# Nested Samplers available in Gleipnir

Gleipnir currently has five classes that can be used to create Nested Sampling objects: one for using a built-in Nesting Sampling implementation and four interfaces to external Nested Sampling packages. In this notebook, we will explore each of these Nested Samplers and how to use them. 

Gleipnir currently has five Nested Sampling classes:

  * NestedSampling - Gleipnir's built-in implementation of Nested Sampling.
  * MultiNestNestedSampling - Wrapper on top of PyMultiNest for running MultiNest
  * PolyChordNestedSampling - Wrapper on top of pypolychord for running PolyChord
  * dyPolyChordNestedSampling - Wrapper on top of dyPolyChord for running dyPolyChord
  * DNest4NestedSampling - Wrapper on top of dnest4 (python bindings of DNest4) for running DNest4
  
We'll cover each one in more detail in the following sections.  

## Model: Egg Carton likelihood

For the purposes of this tutorial, we will again use the Egg Carton likelihood landscape model (as in the ([Intro to Nested Sampling Notebook](https://github.com/LoLab-VU/Gleipnir/blob/master/jupyter_notebooks/Intro_to_Nested_Sampling_with_Gleipnir.ipynb)).  

The model is typically two-dimensional (two parameters) and the landscape generated by the likelihood function is a multi-modal egg carton-like shape; see slide 15 of [this pdf](http://www.nbi.dk/~koskinen/Teaching/AdvancedMethodsInAppliedStatistics2016/Lecture14_MultiNest.pdf) for a visualization of the likelihood landscape. The parameters are each defined on \[0:10pi\] with uniform priors. 

Here is the Egg Carton loglikelihood function which returns the natural logarithm of the likelihood for a given parameter vector:

In [1]:
# Import NumPy
import numpy as np
# Define the loglikelihood function.
def loglikelihood(parameter_vector):
    chi = (np.cos(parameter_vector)).prod()
    return (2. + chi)**5

### Sampled Parameters

Now that we have our loglikelihood function, let's look at how to define parameters for sampling during the Nested Sampling run. The parameters that are sampled are defined by a list of SampledParameter class instances. The SampledParameter class object stores data on the name of the parameter and the parameter's prior probability distribution. 

In [2]:
# Import the SampledParameters class.
from gleipnir.sampled_parameter import SampledParameter

A new SampledParameter needs two arguments: a name and a object defining the prior.

For priors we can use frozen RV objects from scipy.stats; in special cases you could also write your own prior distribution class objects, but for most purposes scipy.stats distributions will be sufficient.

In [3]:
# Let's import the uniform distribution.
from scipy.stats import uniform

In [4]:
# Now we'll create our list sampled parameters.
# There are two parameters 'x' and 'y',each with a uniform prior on [0:10pi]. 
sampled_parameters = list()
sampled_parameters.append(SampledParameter(name='x', prior=uniform(loc=0.0,scale=10.0*np.pi)))
sampled_parameters.append(SampledParameter(name='y', prior=uniform(loc=0.0,scale=10.0*np.pi)))  

Now we've defined our list of paramters that are to be sampled and their prior probability distributions.  We'll use these sampled parameters for each of the Nested Samplers. 

## 1. NestedSampling
The first Nested Sampler is Gleipnir's built-in implementation of the Nested Sampling algorithm, NestedSampling. It is imported from the gleipnir.nested_sampling module. 

In [5]:
from gleipnir.nested_sampling import NestedSampling

This Nested Sampler uses a plug-in style approach to sampling (i.e., replacing dead points) and stopping criterion (i.e., when to exit the Nested Sampling run). These are passed in to the NestedSampling class initialization as optional keyword arguments. Although there are defaults set for these if we don't pass anything in, for the puposes of this tutorial we will go over each and explicitly set them.

### Sampler
The sampler is used in the classic Nested Sampling algorithm to replace dead points during each nested iteration. The samplers are imported from the samplers module:

In [6]:
# Import the sampler we want to use during the Neseted Sampling run.
from gleipnir.samplers import MetropolisComponentWiseHardNSRejection

Currently, Gleipnir just has the one sampler: MetropolisComponentWiseHardNSRejection. This sampler uses a [Metropololis Monte Carlo](http://xbeams.chem.yale.edu/~batista/vaa/node42.html) scheme adapted for Nested Sampling with a hard Nested likelihood level rejection criterion. During the Nested Sampling iteration the most recent dead point is replaced with a survivor that is  then modified using the sampler. More samplers may be added in the future.

Now we can initialize the sampler:

In [7]:
# Initialize the sampler.
sampler = MetropolisComponentWiseHardNSRejection(iterations=10, tuning_cycles=1)

Here the iterations=10 are the number of total component wise update cycles, so the value of 10 will yield 20 component-wise Monte Carlo trial moves (i.e., iterations*(number of sampled parameters)). tuning_cycles=1 sets the number of trial move size tuning cycles; each tuning_cycle is 20 iterations. Note that MetropolisComponentWiseHardNSRejection(10, tuning_cyles=1) is the default value that will be used if we don't explicitly set the sampler.

### Stopping Criterion

The stopping criterion sets the method used to determine when terminate the Nested Sampling iterations. They are imported from the stopping_criterion modulue. There are currenlty two criterion which can be used:
  * NumberOfIterations - Stop after a fixed number of nested iterations.
  * RemainingPriorMass - Stop after the remaining fraction of prior mass is less than or equal to a preset threshold.
Here, we'll use the fixed number of iterations:  

In [8]:
# Import the stopping criterion object. In this case, we'll use a fixed number of iterations.
from gleipnir.stopping_criterion import NumberOfIterations
# Initialize the stopping criterion -- We'll stop after 1000 Nested Sampling iterations.
stopping_criterion = NumberOfIterations(1000)

Note that NumberOfIterations(1000) is the default that will be used by the Nested Sampler if an explicit value is not set.

Now that we've got the sampled parameters, sampler, and stopping criterion, all we need now is the Nested Sampling population size. Let's go ahead and set it:

In [9]:
# Set the NS population size.
population_size=500

Now we create the instance of the NestedSampling class:

In [10]:
NS = NestedSampling(sampled_parameters,
                    loglikelihood,
                    population_size,
                    sampler=sampler,
                    stopping_criterion=stopping_criterion)

Then the Nested Sampling is launched with the run function:

In [11]:
log_evidence, log_evidence_error = NS.run(verbose=False)

Then we can check the output:

In [12]:
print(" Ln(Evidence): {}+-{}".format(log_evidence, log_evidence_error))

 Ln(Evidence): 235.70248428011186+-0.06321396463872658


In addition to the log_evidence and log_evidence_error estimates returned by the run function, the NestedSampling Nested Sampler has the following accesible properties:
  * evidence
  * evidence_error
  * information

In [13]:
evidence = NS.evidence
evidence_error = NS.evidence_error
information = NS.information

The NestedSampling Nested Sampler also has the following functions:
  * posteriors - Estimates of the posterior marginal probability distributions of each parameter. 
  * posterior_moments - Estimates of the first four moments of each posterior marginal probability distribution of the parameters.  
  * akaike_ic - Estimate of the Akaike Information Criterion.
  * bayesian_ic - Estimate of the Bayesian Information Criterion.
  * deviance_ic - Estimate of the Deviance Information Criterion.
  * best_fit_likelihood - Get the maximum likelihood parameter vector.
  * best_fit_posterior - Get highest posterior probability parameter vector.

## 2. MultiNestNestedSampling

Gleipnir also provides a Nested Sampler class, MultiNestNestedSampling, that is an interface to [MultiNest](https://academic.oup.com/mnras/article/398/4/1601/981502); this class is a wrapper on top of [PyMultiNest](https://github.com/JohannesBuchner/PyMultiNest). Use of MultiNest via Gleipnir requires separate building and installation of both MultiNest and the Python wrapper PyMultiNest; see Gleinir's [README](https://github.com/LoLab-VU/Gleipnir#multinest) for the links to those instructions. 

It is imported from the multinest module:

In [14]:
from gleipnir.multinest import MultiNestNestedSampling

Then the instance is created with a similar input pattern as the NestedSampling, but only requires the sampled parameters, loglikelihood, and population size:

In [15]:
MNNS = MultiNestNestedSampling(sampled_parameters, loglikelihood, population_size)

However, the MulitNestNestedSampling object does have some extra keyword arguments that can be passed in to alter the behavior of the MultiNest run (internally passed along to PyMultiNest):
  * importance_nested_sampling (bool): Should MultiNest use
    Importance Nested Sampling (INS). Default: True
  * constant_efficiency_mode (bool): Should MultiNest run in
    constant sampling efficiency mode. Default: False
  * sampling_efficiency (float): Set the MultiNest sampling
    efficiency. 0.3 is recommended for evidence evaluation,
    while 0.8 is recommended for parameter estimation.
    Default: 0.8
  * resume (bool): Resume from a previous MultiNest run (using
    the last saved checkpoint in the MultiNest output files).
    Default: True
  * write_output (bool): Specify whether MultiNest should write
    to output files. True is required for additional
    analysis. Default: True
  * multimodal (bool): Set whether MultiNest performs mode
    separation. Default: True
  * max_mode (int): Set the maximum number of modes allowed in
    mode separation (if multimodal=True). Default: 100
  * mode_tolerance (float): A lower bound for which MultiNest will
    use to separate mode samples and statistics with
    log-evidence value greater the given value.
    Default: -1e90
  * n_clustering_params (int): If multimodal=True, set the number
    of parameters to use in clustering during mode separation.
    If None, then MultiNest will use all the paramters for
    clustering during mode separation. If
    n<(number of sampled parameters), then MultiNest will only
    use a subset composed of the first n parameters for
    clustering during mode separation. Default: None
  * null_log_evidence (float): If multimodal=True, a lower bound
    for which MultiNest can use to separte mode samples and
    statistics with a local log-evidence value greater than the
    given bound. Default: -1.e90
  * log_zero (float): Set a threshold value for which points with
    a loglikelihood less than the given value will be ignored
    by MultiNest. Default: -1e100
  * max_iter (int): Set the maximum number of nested sampling
    iterations performed by MultiNest. If 0, then it is
    unlimited and MultiNest will stop using a different
    criterion. Default: 0

By default, MultiNest will output a set of files with the root 'multinest_run_'. If you want to change the file root you can do so via the multinest_file_root property:

In [16]:
print("Default: {}".format(MNNS.multinest_file_root))
MNNS.multinest_file_root = 'run_eggcarton_multinest_'
print("Changed to: {}".format(MNNS.multinest_file_root))

Default: multinest_run_
Changed to: run_eggcarton_multinest_


Now the MultiNest output files with begin with 'run_eggcarton_multinest'

Once the instance has been defined, you start the Nested Sampling using the run function:

In [17]:
log_evidence, log_evidence_error = MNNS.run(verbose=False)

  analysing data from run_eggcarton_multinest_.txt


In [18]:
print(" Ln(Evidence): {}+-{}".format(log_evidence, log_evidence_error))

 Ln(Evidence): 235.96857829537618+-0.10992362784591564


In addition to the log_evidence and log_evidence_error estimates returned by the run function, the MultiNestNestedSampling Nested Sampler has the following accesible properties:
  * evidence
  * evidence_error
  
The MultiNestNestedSampling Nested Sampler also has the following functions:
  * posteriors - Estimates of the posterior marginal probability distributions of each parameter. 
  * posterior_moments - Estimates of the first four moments of each posterior marginal probability distribution of the parameters.  
  * akaike_ic - Estimate of the Akaike Information Criterion.
  * bayesian_ic - Estimate of the Bayesian Information Criterion.
  * deviance_ic - Estimate of the Deviance Information Criterion.  
  * best_fit_likelihood - Get the maximum likelihood parameter vector.
  * best_fit_posterior - Get highest posterior probability parameter vector.

## 3. PolyChordNestedSampling

Gleipnir also provides a Nested Sampler class, PolyChordNestedSampling, that is an interface to the public version of [PolyChord](https://github.com/PolyChord/PolyChordLite); this class is a wrapper on top of pypolychord, which is itself the python wrapper of PolyChord.  Use of PolyChord via Gleipnir requires separate building and installation of pypolychord; see Gleinir's [README](https://github.com/LoLab-VU/Gleipnir#polychord) for the links to those instructions. 

It is imported from the polychord module:

In [19]:
from gleipnir.polychord import PolyChordNestedSampling

Then the instance is created with a similar input pattern as the NestedSampling, but like the MultiNestNestedSampling object it only requires the sampled parameters, loglikelihood, and population size:

In [20]:
PCNS = PolyChordNestedSampling(sampled_parameters, loglikelihood, population_size)

The PolyChordNestedSampling object does not have any extra keyword arguments to worry about.

By default, PolyChord will output a set of files with the root 'polychord_run_'. If you want to change the file root you can do so via the polychord_file_root property:

In [21]:
print("Default: {}".format(PCNS.polychord_file_root))
PCNS.polychord_file_root = 'run_eggcarton_polychord_'
print("Changed to: {}".format(PCNS.polychord_file_root))

Default: polychord_run
Changed to: run_eggcarton_polychord_


Now the PolyChord output files will begin with 'run_eggcarton_polychord'

Once the instance has been defined, you start the Nested Sampling using the run function:

In [22]:
%%capture
# Run. There is no verbose setting for the PCNS object's run function. 
log_evidence, log_evidence_error = PCNS.run()

In [23]:
print(" Ln(Evidence): {}+-{}".format(log_evidence, log_evidence_error))

 Ln(Evidence): 235.993925103973+-0.111135704580577


In addition to the log_evidence and log_evidence_error estimates returned by the run function, the PolyChordNestedSampling Nested Sampler has the following accesible properties:
  * evidence
  * evidence_error
  
The PolyChordNestedSampling Nested Sampler also has the following functions:
  * posteriors - Estimates of the posterior marginal probability distributions of each parameter. 
  * posterior_moments - Estimates of the first four moments of each posterior marginal probability distribution of the parameters.  
  * akaike_ic - Estimate of the Akaike Information Criterion.
  * bayesian_ic - Estimate of the Bayesian Information Criterion.
  * deviance_ic - Estimate of the Deviance Information Criterion. 
  * best_fit_likelihood - Get the maximum likelihood parameter vector.
  * best_fit_posterior - Get highest posterior probability parameter vector.  

## 4. dyPolyChordNestedSampling

Gleipnir also provides a Nested Sampler class, dyPolyChordNestedSampling, that is an interface to [dyPolyChord](https://github.com/ejhigson/dyPolyChord); this class is a wrapper on top of dyPolyChord, which itself uses the python bindings of PolyChord (i.e., pypolychord). Use of dyPolyChord via Gleipnir requires separate building and installation of pypolychord; see Gleinir's [README](https://github.com/LoLab-VU/Gleipnir#polychord) for the links to those instructions. 

It is imported from the dypolychord module:

In [24]:
from gleipnir.dypolychord import dyPolyChordNestedSampling

Then the instance is created with the common input pattern:

In [25]:
dyPCNS = dyPolyChordNestedSampling(sampled_parameters, loglikelihood, population_size)

Note that the population_size is used as the dyPolyChord parameter `nlive_const`, which is the total computational budget for allocating population sizes during the dynamic Nested Sampling run. The dyPolyChordNestedSampling object does have some extra keyword arguments that can be passed in to alter the behavior of the dyPolyChord run:
   * initial_population_size (int): Set the initial population size for first dyPolyChord exploratory run. Should be < population_size. Default: population_size/2
   * dynamic_goal (float or int): Set the dynamic Nested Sampling goal, which is a number in (0, 1) that determines how to allocate computational effort between parameter estimation and evidence calculation; i.e., with a value of 0 dyPolyChord will focus on parameter estimation only, and opposingly with a value of  1 dyPolyChord will focus on evidence calculation only. Default: 0.5


  

By default, dyPolyChord will output a set of files with the root 'dypolychord_run_' in a 'dypolychord_chains' directory. If you want to change the file root you can do so via the dypolychord_file_root property:

In [26]:
print("Default: {}".format(dyPCNS.dypolychord_file_root))
dyPCNS.dypolychord_file_root = 'run_eggcarton_dypolychord_'
print("Changed to: {}".format(dyPCNS.dypolychord_file_root))

Default: dypolychord_run
Changed to: run_eggcarton_dypolychord_


Now the dyPolyChord output files will begin with 'run_eggcarton_dypolychord'

Once the instance has been defined, you start the Nested Sampling using the run function:

In [27]:
%%capture
# Run. There is no verbose setting for the dyPCNS object's run function. 
log_evidence, log_evidence_error = dyPCNS.run()

In [28]:
print(" Ln(Evidence): {}+-{}".format(log_evidence, log_evidence_error))

 Ln(Evidence): 235.8615038671045+-None


Note that dyPolyChord doesn't return a rough estimate of the error in the evidence, it instead suggests using a bootstrap analysis of multiple dyPolyChord runs via nestcheck to estimate the error. At the moment, Gleipnir does not implement an interface for this analysis, so users must do that on their own if they want the bootstrapped error estimates.

In addition to the log_evidence estimate returned by the run function, the dyPolyChordNestedSampling Nested Sampler has the following accesible properties:
  * evidence
  * evidence_error --> Just returns None
  
The dyPolyChordNestedSampling Nested Sampler also has the following functions:
  * posteriors - Estimates of the posterior marginal probability distributions of each parameter. 
  * posterior_moments - Estimates of the first four moments of each posterior marginal probability distribution of the parameters.
  * akaike_ic - Estimate of the Akaike Information Criterion.
  * bayesian_ic - Estimate of the Bayesian Information Criterion.
  * deviance_ic - Estimate of the Deviance Information Criterion. 
  * best_fit_likelihood - Get the maximum likelihood parameter vector.
  * best_fit_posterior - Get highest posterior probability parameter vector.  

## 4. DNest4NestedSampling

Gleipnir also provides a Nested Sampler class, DNest4NestedSampling, that is an interface to [DNest4](https://github.com/eggplantbren/DNest4); this class is a wrapper on top of dnest4, which is itself the python wrapper of DNest4. Use of DNest4 via Gleipnir requires separate building and installation of DNest4 and its Python wrapper; see Gleinir's [README](https://github.com/LoLab-VU/Gleipnir#dnest4) for the links to those instructions. 

It is imported from the dnest4 module:

In [29]:
from gleipnir.dnest4 import DNest4NestedSampling

Then the instance is created with a similar input pattern as the NestedSampling, but only requires the sampled parameters, loglikelihood, and population size:

In [30]:
DNS = DNest4NestedSampling(sampled_parameters, loglikelihood, population_size)

However, the DNest4NestedSampling object does have some extra keyword arguments that can be passed in to alter the behavior of the DNest4 run:
  * n_diffusive_levels (int): Set the maximum number of
      diffusive likelihood levels for DNest4 to use.
      Default: 20
  * dnest4_backend (str : "memory" or "csv"): Set which
      DNest4 backend to use. "memory" means outputs are 
      just kept in memory. "csv" means outputs are 
      written to disk in files with a csv format. Default: "memory"
  * num_steps (int): The number of nested iterations to run. If None, 
      will run forever. 
      Default: 1000
  * new_level_interval (int): The number of nested iterations to run before
     creating a new diffusive likelihood level. Default: 10000
  * lam (float): Set the backtracking scale length. Default: 5.0
  * beta (float): Set the strength of effect to force the histogram
      to equal bin counts. Default: 100

Let's re-initialize the sampler and set the num_steps variable (the default value is 1000).

In [31]:
DNS = DNest4NestedSampling(sampled_parameters, loglikelihood, population_size, num_steps=500)

If the dnest4_backend keyword argument is set to "csv" (the default is "memory", which has no output files) then the DNest4 will output a set of files with the root 'dnest4_run_'. If you want to change the file root you can do so via the dnest4_file_root property:

In [32]:
print("Default: {}".format(DNS.dnest4_file_root))
DNS.dnest4_file_root = 'run_eggcarton_dnest4_'
print("Changed to: {}".format(DNS.dnest4_file_root))

Default: ./dnest4_run_
Changed to: run_eggcarton_dnest4_


Now any DNest4 output files would begin with 'run_eggcarton_dnest4'

Once the instance has been defined, you start the Nested Sampling using the run function:

In [33]:
log_evidence, log_evidence_error = DNS.run(verbose=False)

500
500


In [34]:
print(" Ln(Evidence): {}+-{}".format(log_evidence, log_evidence_error))

 Ln(Evidence): 235.82229164699086+-0.11094648091550836


In addition to the log_evidence and log_evidence_error estimates returned by the run function, the DNest4NestedSampling Nested Sampler has the following accesible properties:
  * evidence
  * evidence_error
  * information
  
The DNest4NestedSampling Nested Sampler also has the following functions:
  * posteriors - Estimates of the posterior marginal probability distributions of each parameter. 
  * posterior_moments - Estimates of the first four moments of each posterior marginal probability distribution of the parameters.
  * akaike_ic - Estimate of the Akaike Information Criterion.
  * bayesian_ic - Estimate of the Bayesian Information Criterion.
  * deviance_ic - Estimate of the Deviance Information Criterion.  
  * best_fit_likelihood - Get the maximum likelihood parameter vector.
  * best_fit_posterior - Get highest posterior probability parameter vector.  